<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Domi-DEV/tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install on colab
# downgrade accelerate to make it compatible with Trainer API, see https://github.com/huggingface/accelerate/releases
!pip install accelerate==0.27.2
# after 4.38.2, top_k_top_p_filtering is removed, see https://github.com/huggingface/trl/issues/6
!pip install -U transformers==4.38.2
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [8]:
import json
import pandas as pd

In [3]:
url = "https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire1.json?token=GHSAT0AAAAAACY3NQSAVESXWHFGJM3MFGEWZ32PBRQ"
Q1 = pd.read_json(url)
print(Q1)


HTTPError: HTTP Error 404: Not Found

### Converting answer options of multi select
In this block of code we are trying to convert the options of the questions posed in the questionnaires to a format, so that every combination of options is one option of the question.
For more clarity, let us consider a question with possible answers 'a' and 'b'. In the converted version, the options of that multi select questions are [], ['a'], ['b'], ['a', 'b'].

In [9]:
import json
import urllib
from itertools import chain, combinations

questions = []

for i in range (1, 6):
    url = f"https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire{i}.json"
    response = urllib.request.urlopen(url)
    q = json.loads(response.read())
    for question in q:
      if question['type'] != 'MULTI_SELECT':
        continue
      else:
        options_json = question['options']
        options_list = [option['option'] for option in options_json]
        powerset = chain.from_iterable(combinations(options_list, r) for r in range(len(options_list)+1))
        question['options'] = [', '.join(list(s)) for s in powerset]
    questions = questions + q






**To be modfied:** load questions into json in the github repo

In [6]:
questions


[{'id': 'aa2d8cdd-0758-4035-b0b6-ca18e2f380d8',
  'type': 'SINGLE_SELECT',
  'question': 'Data processing consent',
  'options': [{'id': 'ee0437c0-6335-4b88-8bc5-d4eb8e2c68bf', 'option': 'Yes'},
   {'id': 'd357ab84-929f-440a-b9ad-42ff36402a53', 'option': 'No'}]},
 {'id': '12e1ed1d-edaa-4e93-8645-de3850e998f9',
  'type': 'SINGLE_SELECT',
  'question': 'Customer group',
  'options': [{'id': '53cc44fa-397b-40bf-8af8-3f629818c93b',
    'option': 'End User'},
   {'id': 'bab850d1-2de8-4cae-9f87-2c86d41615e8',
    'option': 'Wholesaler, Distributor'},
   {'id': '31ffae1e-f742-4a0b-b279-eb393483f075',
    'option': 'Consultant, Planner, Architect'},
   {'id': 'ed56f777-a7cd-48db-9d36-e2579c57e361', 'option': 'R&D'}]},
 {'id': '625012ae-9192-4cf6-a73d-55e1813d6014',
  'type': 'MULTI_SELECT',
  'question': 'Products interested in',
  'options': ['',
   'MY-SYSTEM',
   'Notion',
   'JTS',
   'JS EcoLine',
   'AKW100',
   'AX100',
   'MY-SYSTEM, Notion',
   'MY-SYSTEM, JTS',
   'MY-SYSTEM, JS EcoL

In [ ]:
with open('q_changed_options.json', 'w') as fp:
    json.dump(questions, fp)

In [ ]:
q_products_interested_in = [question for question in questions if question['id']== 'e95e34ce-1758-4037-9474-99d0557a7a29']
q_products_interested_in = q_products_interested_in[0]

q_products_interested_in['context'] = 'The contact person expressed interest in both Data Enrichment and Visit Report for their upcoming projects.'
q_products_interested_in['answer'] = "DataEnrichment, 'VisitReport"
q_products_interested_in['question'] = 'In which products was the contact person interested in?'

In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name)


question = q_products_interested_in['question']
options = q_products_interested_in['options']
context = q_products_interested_in['context']

# Prepare inputs for multiple choice
inputs = tokenizer(
    [[context, question + " " + str(option)] for option in options],
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Reshape inputs to match the expected format of the model
input_ids = inputs["input_ids"].reshape(1, len(options), -1)
attention_mask = inputs["attention_mask"].reshape(1, len(options), -1)

# Forward pass through the model
outputs = model(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.logits  # Shape: [batch_size, num_choices]

# Predict the option with the highest score
predicted_option = torch.argmax(logits, dim=1).item()

print(f"The predicted answer is: {options[predicted_option]}")


In [ ]:
# Print input IDs as strings
for idx, input_ids in enumerate(inputs["input_ids"]):
    print(f"Option {idx + 1}:")
    print(tokenizer.decode(input_ids, skip_special_tokens=True))
    print("-" * 50)

## Creating some neccesary functions

In [ ]:
# print model name of model
print(f"The model name: {model.name_or_path}")

In [ ]:

from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch
'''
model_output -> creates output for every question in the dataset and safes it in a list of dicts. One dic has keys 'answer', 'predicted_answer', 'type'
parameters:
- model: one hugging face model
- tokenizer: hugging face tokenizer
- questions: QA-dataset in json format
'''
def model_output(model, tokenizer, questions):
  answer_comparison = []
  for question in questions:
        context = question['context']
        question_text = question['question']
        options = question['options']
        answer = question['answer']
        question_type = question['type']

        # Prepare inputs for multiple choice
        inputs = tokenizer(
            [[context, question_text + " " + str(option)] for option in options],
            return_tensors="pt",
            padding=True,
            truncation=True
        )

        # Reshape inputs to match the expected format of the model
        input_ids = inputs["input_ids"].reshape(1, len(options), -1)
        attention_mask = inputs["attention_mask"].reshape(1, len(options), -1)

        # Forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: [batch_size, num_choices]

        # Predict the option with the highest score
        predicted_option = torch.argmax(logits, dim=1).item()

        answer_comparison.append({'answer': answer, 'predicted_answer': options[predicted_option], 'type': question_type})
  return answer_comparison

'''
accuracy -> computes the total accuracy and accuracy for each question type for the passed list of dicts. One dict in the list is one question with keys 'answer', 'predicted_answer', 'type'
parameters:
list of dicts with entries...
...predicted answer
...answer
...type of question
'''
def accuracy(answer_comparison):
    correct_multi_select = 0
    correct_single_select = 0
    correct_text = 0
    correct_number = 0
    correct_date = 0
    correct_total = 0
    total = 0

    for entry in answer_comparison:
        question_type = entry['type']
        if entry['answer'] == entry['predicted_answer']:
            if question_type == 'MULTI_SELECT':
                correct_multi_select += 1
                total_multi_select += 1
            elif question_type == 'SINGLE_SELECT':
                correct_single_select += 1
                total_single_select += 1
            elif question_type == 'TEXT':
                correct_text += 1
                total_text += 1
            elif question_type == 'NUMBER':
                correct_number += 1
                total_number += 1
            elif question_type == 'DATE':
                correct_date += 1
                total_date += 1
            else:
              continue
            correct_total += 1
        total += 1
    accuracy_total = correct_total / total
    accuracy_multi_select = correct_multi_select / total_multi_select
    accuracy_single_select = correct_single_select / total_single_select
    accuracy_text = correct_text / total_text
    accuracy_number = correct_number / total_number
    accuracy_date = correct_date / total_date
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date
'''
print_out_model_quality: takes the computations of function accuracy() and prints them out
parameters:
- accuracy_total
- accuracy_multi_select
- accuracy_single_select
- accuracy_text
- accuracy_number
- accuracy_date
'''
def print_out_model_quality(accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date):
    accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date = accuracy(model, tokenizer, questions)
    print(f"""Accuracy values of model: {model.name_or_path}\n
    Total: {accuracy_total}\n
    Multi-select: {accuracy_multi_select}\n
    Single-select: {accuracy_single_select}\n
    Text: {accuracy_text}\n
    Number: {accuracy_number}\n
    Date: {accuracy_date}\n""")
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date




Here, we try out how to handle the answering process of the models that we have to evaluate

In [10]:
import json
import requests

url = f"https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/datasets/qa_dataset.json"
response = urllib.request.urlopen(url)
q = json.loads(response.read())
q

[{'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['Adito'],
  'context': 'Oh, CRM systems?  I use Adito.  It works well for me.'},
 {'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['Salesforce'],
  'context': "Oh, CRM systems? I use Salesforce.  It's pretty good, actually."},
 {'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['Close.io'],
  'context': "Oh, CRM systems? I use Close.io.  It's pretty good for my needs."},
 {'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['SAP Sales Cloud'],
  'context': 'Oh, CRM systems? I use SAP Sales Cloud.  It works pretty well for me.'},
 {'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['CAS'],
  'context': "Oh, a CRM system?  I use CAS. It's pretty straightforward for my needs."},
 {'question': 'CRM-System',
  'type': 'SINGLE_SELECT',
  'options': ['Pipedrive'],
  'context': 'Oh, I use Pipedrive.  It works pretty well for me.'},
 {'question': 'Customer grou

In [17]:
import numpy as np


def prediction_probabilities(predictions):

    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    all_scores = predictions.start_logit+predictions.end_logit
    return softmax(all_scores)


In [20]:
from transformers import pipeline

answer_generator = pipeline(task="question-answering", model="deepset/roberta-base-squad2")
for i in range(5):
  result_type = answer_generator(question=q[i]['question'], context=f"{q[i]['context']} {q[i]['options']}")
  print(f"Result for question {i}: Intended answer: {q[i]['options']}, Answer: {result_type['answer']}, Score: {result_type['score']}")

Result for question 0: Intended answer: ['Adito'], Answer: Adito, Score: 0.0002729370316956192
Result for question 1: Intended answer: ['Salesforce'], Answer: Salesforce, Score: 0.00014287990052253008
Result for question 2: Intended answer: ['Close.io'], Answer: Close.io, Score: 1.2193746442790143e-05
Result for question 3: Intended answer: ['SAP Sales Cloud'], Answer: SAP Sales Cloud, Score: 0.00010010084952227771
Result for question 4: Intended answer: ['CAS'], Answer: CAS, Score: 0.006240698974579573
